In [ ]:

from tavily import TavilyClient
from langgraph.graph import Graph
from langchain.prompts import PromptTemplate
from requests.exceptions import HTTPError


In [ ]:
#in this step we are going to use our api key(change it with yours)
API_KEY = "tvly-dev"#use your api key
tavily = TavilyClient(api_key=API_KEY)

In [ ]:

def research_agent(inputs):
    try:
        query = inputs["query"]  # Extract the query correctly
        print(f"Sending query to Tavily: {query}")

        response = tavily.search(query=query, max_results=5, search_depth="basic")

        if response and 'results' in response:
            print(f"Tavily API response: {response}")
            return {"data": response['results']}
        else:
            print("No results found in Tavily API response.")
            return {"data": []}
    except HTTPError as e:
        print(f"Tavily API error: {e}")
        return {"data": []}

In [ ]:

prompt_template = PromptTemplate(
    input_variables=["data"],
    template="Based on the following information, draft a detailed answer:\n\n{data}"
)


In [ ]:
#  Drafter Agent
def answer_drafter_agent(inputs):
    data = inputs["data"]
    if not data:
        return {"answer": "No data available to draft an answer."}

    
    prompt = prompt_template.format(data=data)


    answer = "Here is the information I found:\n\n"
    for i, result in enumerate(data, start=1):
        answer += f"{i}. **{result['title']}**\n"
        answer += f"   - URL: {result['url']}\n"
        answer += f"   - Summary: {result['content']}\n\n"

    return {"answer": answer}


In [ ]:
# Initialize the Lang graph workflow
workflow = Graph()


workflow.add_node("research", research_agent)
workflow.add_node("draft", answer_drafter_agent)


workflow.add_edge("research", "draft")


workflow.set_entry_point("research")
workflow.set_finish_point("draft")

compiled_workflow = workflow.compile()


def run_workflow(query):
    return compiled_workflow.invoke({"query": query})

In [52]:
# Main Execution
if __name__ == "__main__":
    query = "full form of usa"
    final_answer = run_workflow(query)
    print(final_answer["answer"])

Sending query to Tavily: full form of usa
Tavily API response: {'query': 'full form of usa', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'USA Full Form - What Is Full Form', 'url': 'https://www.whatisfullform.com/usa-full-form/', 'content': 'The FULL FORM of USA is United States of America Its resource is Washington, D.C. It is the fourth-largest country in the world after Russia, Canada, as well as China. It is twice the size of the European Union. The currency of the USA is the United States dollar (USD). It has no official language, however, English is spoken throughout the nation.', 'score': 0.9152729, 'raw_content': None}, {'title': 'What is the full form of USA?', 'url': 'https://fullform.blog/countries/usa-full-form/', 'content': 'USA: United States of America. The full form of USA is "United States of America". The United States of America (USA) is a country of North America, a federal republic of 50 states, five major autonomous territories